In [1]:
import pandas as pd
import numpy as np
from IPython.display import display
from PSyn import matrix_functions
from PSyn import brain
import os
import time
SOURCE_DATA_PATH = 'data/task1/dev/'
from collections import Counter

In [8]:
source_files = os.listdir(SOURCE_DATA_PATH)
source = 'polish-train-high'
# source_files = ['polish-train-medium']

start = time.time()
# for source in source_files:
print(source)
prf, total_corect, global_attr, total_entries, correct_inflections, macro_prf = brain.test_model_accuracy(source, classifier='linear_regression')
# print(time.time() - start)
start = time.time()
print('Macro PRF:', )
print('Guessed %d inflections correctly out of %d.\nAccuracy: %f' % (total_corect, total_entries, (total_corect * 100 / float(total_entries))))

polish-train-high
Total Alphabets:
{'b', 'p', 'ś', 'f', 'ł', 'y', 'M', 'k', 'W', 'r', 'ę', 'd', 'm', 'B', 'R', 'w', 'i', 'n', 'ó', 'g', 'Ż', 'h', 'ą', ' ', 'u', 'j', 'z', 'c', 'ć', 'a', 'C', 'e', 'ż', 'G', 'ź', 't', 'l', 'ń', 's', 'o'}
Epsilon: 9.000000
Ci: 40.000000


/home/gaurav/anaconda3/lib/python3.6/site-packages/numpy/lib/function_base.py:1110: RuntimeWarning: Mean of empty slice.
  avg = a.mean(axis)
/home/gaurav/anaconda3/lib/python3.6/site-packages/numpy/core/_methods.py:80: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)


ZeroDivisionError: float division by zero

In [ ]:
source_files = os.listdir(SOURCE_DATA_PATH)
source = 'polish-train-high'
# source_files = ['polish-train-medium']

start = time.time()
# for source in source_files:
print(source)
prf, total_corect, global_attr, total_entries, correct_inflections, macro_prf = brain.test_model_accuracy(source, classifier='random_forest')
# print(time.time() - start)
start = time.time()
print('Macro PRF:', )
print('Guessed %d inflections correctly out of %d.\nAccuracy: %f' % (total_corect, total_entries, (total_corect * 100 / float(total_entries))))

In [ ]:
global_attr

In [ ]:
source_csv = open('data/task1/dev/polish-dev', 'r')
dict_for_df = {'source': [], 'target': [], 'source_node': [], 'pos': []}
content = source_csv.readlines()
for line in content:
    row = line.split('\t')
    dict_for_df['source'].append(row[0])
    dict_for_df['target'].append(row[1])
    dict_for_df['source_node'].append(
        row[0] + '-' + row[2].strip().replace(';', '_'))
    dict_for_df['pos'].append(row[2].split(';')[0])
source_df = pd.DataFrame.from_records(dict_for_df)
source_df = source_df[source_df['pos'] == 'N']
idx = source_df.index[source_df['source_node'] == 'Cezar-N_DAT_PL']
list(source_df.loc[idx]['target'])

In [ ]:
o = pd.DataFrame.from_dict(correct_inflections)
ops_ = {}
for key in o.columns.values:
    ops_[key] = []
    for i, _ in enumerate(o[key]):
        if _:
            ops_[key].append(o[key].index.values[i])
t = []
for s_node in list(ops_.keys()):
    idx = source_df.index[source_df['source_node'] == s_node]
    t.append(list(source_df.loc[idx]['target'])[0])
pd.DataFrame.from_dict({'source': list(ops_.keys()), 'opn_sequence': list(ops_.values()), 'target': t})

In [ ]:
prf_values = list(prf.values())
prf_keys = list(prf.keys())
prf_df = pd.DataFrame.from_dict({'operation': [_.split('_')[0] for _ in prf_keys],
                                 'char': [_.split('_')[1] for _ in prf_keys],
                                 'lpos': [_.split('_')[2] for _ in prf_keys],
                                 'rpos': [_.split('_')[3] for _ in prf_keys],
                                 'precision': [_[0] for _ in prf_values],
                                 'recall': [_[1] for _ in prf_values],
                                 'f_score': [_[2] for _ in prf_values]})
display(prf_df.head())
P = [ e[0] for e in list(prf.values())]
R = [ e[1] for e in list(prf.values())]
F = [ e[2] for e in list(prf.values())]
print(np.mean(P), np.mean(R), np.mean(F))

In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt
print('Model-wise boxplots for full-length bitstrings')
plt.clf()
box_plot = sns.boxplot(x='operation', y='precision', data=prf_df)
display(box_plot.get_figure())
plt.clf()
box_plot = sns.boxplot(x='operation', y='recall', data=prf_df)
display(box_plot.get_figure())
plt.clf()
box_plot = sns.boxplot(x='operation', y='f_score', data=prf_df)
display(box_plot.get_figure())
plt.clf()
box_plot = sns.boxplot(x='char', y='precision', data=prf_df)
display(box_plot.get_figure())

In [ ]:
# Load graphs of classifiers
import pickle
import os
from sklearn import tree
LABELS_PATH = 'data/task1/output/labels/'
labels = pickle.load(open(LABELS_PATH + 'polish-train-high.p', 'rb'))
clf_path = 'data/task1/output/prediction/polish-train-high/'
features = pickle.load(open('feature_map_polish.p', 'rb')).keys()
dtrees = [pickle.load(open(clf_path + clf, 'rb')) for clf in os.listdir(clf_path)]
for i, t in enumerate(os.listdir(clf_path)):
    if 'decision_tree.p' not in t:
        continue
    tree.export_graphviz(dtrees[i], out_file='dotfiles/' + t.replace('.p', '.dot'), feature_names=list(features))

In [ ]:
from subprocess import check_call
for dot_f in os.listdir('dotfiles/'):
    check_call(['dot','-Tpng',('dotfiles/' + dot_f),'-o','pngs/' + dot_f.replace('.dot', '.png')])

In [ ]:
for i, t in enumerate(os.listdir(clf_path)):
    if 'decision_tree.p' in t:
        continue
    feature_importances = dtrees[i].feature_importances_
    
    